# Energy Forecasting - Model Training

In this notebook:
- Load preprocessed hourly power consumption data
- Create time-based features
- Train regression models (Random Forest, LightGBM)
- Evaluate model performance

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

df = pd.read_csv('../data/processed/hourly_power_consumption.csv', parse_dates=['Datetime'])
df.set_index('Datetime', inplace=True)

df['hour'] = df.index.hour
df['day'] = df.index.day
df['weekday'] = df.index.weekday
df['month'] = df.index.month

df = df.dropna(subset=['Global_active_power'])

y = df['Global_active_power']
X = df[['hour', 'day', 'weekday', 'month']]

X_train, X_test, y_train, y_test, = train_test_split(X,y,test_size=0.2,shuffle=False)

model = LGBMRegressor(n_estimators=100,learning_rate=0.1,random_state=42)
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

joblib.dump(model,'model.pkl')

results_df = pd.DataFrame({
    'actual': y_test,
    'predicted': y_pred
}).dropna()

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE: {mae:.3f},RMSE:{rmse:.3f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000398 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 76
[LightGBM] [Info] Number of data points in the train set: 27334, number of used features: 4
[LightGBM] [Info] Start training from score 1.110571
MAE: 0.440,RMSE:0.603


In [2]:
plt.figure(figsize=(14, 5))
plt.plot(y_test.values[:200], label='Actual')
plt.plot(y_pred[:200], label='Predicted (LightGBM)')
plt.title("Hourly Power Consumption Forecast - First 200 Hours")
plt.xlabel("Time(hours)", fontsize=12)
plt.ylabel("Global Active Power (kW)", fontsize=12)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()